In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib

E:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
import numpy as np
import pandas as pd
df = pd.read_csv('sonar/sonar.all-data', header = None)
df=df.replace('R',0)
df=df.replace('M',1)
data=df.values
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(data[:,0:60],data[:,60], test_size=test_size, random_state=seed)
y_train=y_train-1
y_test=y_test-1

In [10]:
# 模型训练，使用GBDT算法
gbr = GradientBoostingClassifier(n_estimators=30000, max_depth=2, min_samples_split=2, learning_rate=0.01)
gbr.fit(X_train, y_train.ravel())
joblib.dump(gbr, 'train_model_result4.m')   # 保存模型

['train_model_result4.m']

In [11]:
y_gbr = gbr.predict(X_train)
y_gbr1 = gbr.predict(X_test)
acc_train = gbr.score(X_train, y_train)
acc_test = gbr.score(X_test, y_test)
print(acc_train)
print(acc_test)

1.0
0.8405797101449275


In [12]:
# AdaBoostClassifier
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=200, learning_rate=0.8)

In [13]:
bdt.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=2,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=5,
                                                         min_samples_split=20,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                              

In [14]:
y_bdt = bdt.predict(X_train)
y_bdt = bdt.predict(X_test)
acc_train = bdt.score(X_train, y_train)
acc_test = bdt.score(X_test, y_test)
print(acc_train)
print(acc_test)

1.0
0.8260869565217391


In [15]:
#  xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
model = XGBClassifier(learning_rate=0.01,max_depth=10)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="mlogloss", eval_set=eval_set, verbose=True)

XGBoostError: [11:04:22] src/metric/multiclass_metric.cu:32: Check failed: label_error >= 0 && label_error < static_cast<int32_t>(n_class): MultiClassEvaluation: label must be in [0, num_class), num_class=1 but found 1 in label

In [ ]:
plot_importance(model)
pyplot.show()

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
esb_clf2=VotingClassifier(estimators=[
    ('Xgboost',model),
    ('AdaBoost',bdt),
    ('GBDT',gbr)],
    weights=[1,1,1],
    voting='hard')

In [ ]:
esb_clf2.fit(X_train,y_train)
esb_pred2=esb_clf2.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report as report
print('ensemble-hard:\n',report(y_test,esb_pred2))
print(y_test)
print(esb_pred2)

In [ ]:
df = pd.read_csv('data/cmc_train.csv', header = None)
data=df.values[1:]
X_train=data[:,[0,3,7,6]]
y_train=data[:,9]
y_train=y_train.astype(int)

df = pd.read_csv('data/cmc_test.csv', header = None)
data=df.values[1:]
data=data.astype(float)
X_test=data[:,[0,3,7,6]]
y_test=data[:,9]
y_test=y_test.astype(int)

In [ ]:
# 模型训练，使用GBDT算法
gbr = GradientBoostingClassifier(n_estimators=30000, max_depth=2, min_samples_split=2, learning_rate=0.01)
gbr.fit(X_train, y_train.ravel())
joblib.dump(gbr, 'train_model_result4.m')   # 保存模型

In [ ]:
y_gbr = gbr.predict(X_train)
y_gbr1 = gbr.predict(X_test)
acc_train = gbr.score(X_train, y_train)
acc_test = gbr.score(X_test, y_test)
print(acc_train)
print(acc_test)
print(y_gbr1)
print(y_test)

In [ ]:
# AdaBoostClassifier
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=200, learning_rate=0.1)

In [ ]:
bdt.fit(X_train,y_train)

In [ ]:
y_bdt = bdt.predict(X_train)
y_bdt = bdt.predict(X_test)
acc_train = bdt.score(X_train, y_train)
acc_test = bdt.score(X_test, y_test)
print(acc_train)
print(acc_test)
print(y_gbr1)
print(y_test)

In [ ]:
model = XGBClassifier(learning_rate=0.09,max_depth=10)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="mlogloss", eval_set=eval_set, verbose=True)

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
rf_clf=RandomForestClassifier(
    n_estimators=21,
    criterion="gini",
    max_depth=4,
    random_state=0)
rf_clf.fit(X_train,y_train)

In [ ]:
rf_pred=rf_clf.predict(X_test)
print('rf:\n',report(y_test, rf_pred))
print(y_test)
print(rf_pred)

In [ ]:
esb_clf2=VotingClassifier(estimators=[
    ('Xgboost',model),
    ('AdaBoost',bdt),
    ('GBDT',gbr),
    ('RF',rf_clf)],
    weights=[1,0.4,1,0.4],
    voting='hard')

In [ ]:
esb_clf2.fit(X_train,y_train)
esb_pred2=esb_clf2.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report as report
print('ensemble-hard:\n',report(y_test,esb_pred2))
print(y_test)
print(esb_pred2)